In [1]:
library(twang)
library(weights)
library(Matching)
library(tidyverse)

Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: Hmisc
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘ggplot2’

The following object is masked from ‘package:latticeExtra’:

    layer


Attaching package: ‘Hmisc’

The following objects are masked from ‘package:xtable’:

    label, label<-

The following object is masked from ‘package:survey’:

    deff

The following objects are masked from ‘package:base’:

    format.pval, units

Loading required package: gdata
gdata: read.xls suppo

In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,1,1,0,0,0,0,1,0.2804613,1.389779
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,1,1,0,0,0,0,0,0.2244447,1.289399
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602


In [4]:
library(tableone)

In [5]:
features <- c("age", "gender", "first_careunit", "weight", "elix_score",
              "saps", "sofa", "vent", "vaso", "sedative",
              "icd_chf", "icd_afib", "icd_renal", "icd_liver",
              "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy", "icu_adm_weekday",
              "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_flag",
              "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
              "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
              "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
              "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
              "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

In [6]:
tab <- CreateTableOne(vars = features,
                      argsNormal = list(var.equal = FALSE),
                      strata = "echo",
                      data = full_data,)
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [7]:
tab_df

,0,1,p,SMD
n,3099,3262,,
age (mean (sd)),66.69 (17.21),65.74 (16.55),0.026,0.056
gender = F (%),1575 (50.8),1558 (47.8),0.016,0.061
first_careunit = MICU (%),2404 (77.6),2593 (79.5),0.067,0.047
weight (mean (sd)),78.56 (23.58),83.17 (26.87),<0.001,0.182
elix_score (mean (sd)),8.51 (7.45),10.07 (7.67),<0.001,0.207
saps (mean (sd)),19.63 (5.79),20.76 (5.44),<0.001,0.202
sofa (mean (sd)),5.31 (3.62),6.33 (3.79),<0.001,0.277
vent = 1 (%),1459 (47.1),1901 (58.3),<0.001,0.226
vaso = 1 (%),839 (27.1),1228 (37.6),<0.001,0.227


In [8]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign.rds"))

In [9]:
tab_wtd <- svyCreateTableOne(vars = features, strata = "echo", data = ipw_svydesign)

In [10]:
capture.output(tab_wtd_df <- tab_wtd %>%
               print %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [11]:
tab_wtd_df

,0,1,p
n,5739.48,5842.36,
age (mean (sd)),65.92 (16.61),65.02 (16.83),0.076
gender = F (%),2824.2 (49.2),2824.0 (48.3),0.523
first_careunit = MICU (%),4473.6 (77.9),4641.0 (79.4),0.178
weight (mean (sd)),82.12 (25.47),82.70 (26.86),0.163
elix_score (mean (sd)),9.53 (7.60),9.90 (7.68),0.114
saps (mean (sd)),22.08 (5.49),22.18 (5.10),0.134
sofa (mean (sd)),6.99 (4.05),7.28 (3.81),0.038
vent = 1 (%),2907.8 (50.7),3139.0 (53.7),0.025
vaso = 1 (%),1759.7 (30.7),1965.5 (33.6),0.020


In [12]:
ps_icustay_id <- data.table::fread(file.path(data_dir, "ps_icustay_id"), data.table = FALSE) %>% pull(icustay_id)

In [13]:
ps_df <- full_data %>% filter(icustay_id %in% ps_icustay_id)

In [14]:
tab_ps <- CreateTableOne(vars = features,
                         strata = "echo",
                         argsNormal = list(var.equal = FALSE),
                         data = ps_df)

In [15]:
capture.output(tab_ps_df <- tab_ps %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [16]:
tab_ps_df

,0,1,p,SMD
n,1626,1626,,
age (mean (sd)),66.58 (16.54),65.84 (17.07),0.212,0.044
gender = F (%),780 (48.0),797 (49.0),0.575,0.021
first_careunit = MICU (%),1274 (78.4),1283 (78.9),0.732,0.014
weight (mean (sd)),81.35 (24.69),79.50 (23.11),0.035,0.077
elix_score (mean (sd)),9.54 (7.67),8.82 (7.52),0.006,0.096
saps (mean (sd)),20.29 (5.42),19.80 (5.17),0.008,0.092
sofa (mean (sd)),5.99 (3.74),5.55 (3.50),<0.001,0.123
vent = 1 (%),851 (52.3),806 (49.6),0.123,0.055
vaso = 1 (%),531 (32.7),507 (31.2),0.387,0.032


In [17]:
wtd_pval <- tab_wtd_df %>%
setNames(c("trtd", "ctrl", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = strsplit(cov, " ") %>% map_chr(first)) %>%
filter(!is.null(pval) & !is.na(pval) & pval != "") %>%
mutate(pval = str_replace_all(pval, "<", "")) %>%
mutate(pval = as.numeric(pval)) %>%
select(cov, pval)

wtd_pval

cov,pval
age,0.076
gender,0.523
first_careunit,0.178
weight,0.163
elix_score,0.114
saps,0.134
sofa,0.038
vent,0.025
vaso,0.020
sedative,0.030


In [18]:
tab_fmt <- tab_df %>%
setNames(c("trtd", "ctrl", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_ = strsplit(cov, " ") %>% map_chr(first)) %>%
mutate(cov_fmt = str_replace_all(cov_, "_", " ")) %>%
mutate(cov_fmt = str_replace_all(cov_fmt, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_fmt = case_when(
    grepl("icd|vs|lab|day", cov_) ~ toupper(cov_fmt),
    TRUE ~ tools::toTitleCase(cov_fmt)
)) %>%
mutate(ctrl = ifelse(grepl("%", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse(grepl("%", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov_fmt, ctrl, trtd, pval, smd) %>%
setNames(c("Covariate", "Non-Echo", "Echo", "p value", "SMD"))

tab_fmt

Covariate,Non-Echo,Echo,p value,SMD
n,3262,3099,,
Age,65.74 (16.55),66.69 (17.21),0.026,0.056
Gender,47.8%,50.8%,0.016,0.061
Careunit,79.5%,77.6%,0.067,0.047
Weight,83.17 (26.87),78.56 (23.58),<0.001,0.182
Elix Score,10.07 (7.67),8.51 (7.45),<0.001,0.207
Saps,20.76 (5.44),19.63 (5.79),<0.001,0.202
Sofa,6.33 (3.79),5.31 (3.62),<0.001,0.277
Vent,58.3%,47.1%,<0.001,0.226
Vaso,37.6%,27.1%,<0.001,0.227


In [19]:
tab_ps_fmt <- tab_ps_df %>%
setNames(c("trtd", "ctrl", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_ = strsplit(cov, " ") %>% map_chr(first)) %>%
mutate(cov_fmt = str_replace_all(cov_, "_", " ")) %>%
mutate(cov_fmt = str_replace_all(cov_fmt, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_fmt = case_when(
    grepl("icd|vs|lab|day", cov_) ~ toupper(cov_fmt),
    TRUE ~ tools::toTitleCase(cov_fmt)
)) %>%
mutate(ctrl = ifelse(grepl("%", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse(grepl("%", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov_fmt, ctrl, trtd, pval, smd) %>%
setNames(c("Covariate", "Non-Echo (PS matching)", "Echo (PS matching)", "p value (PS matching)", "SMD (PS matching)"))

tab_ps_fmt

Covariate,Non-Echo (PS matching),Echo (PS matching),p value (PS matching),SMD (PS matching)
n,1626,1626,,
Age,65.84 (17.07),66.58 (16.54),0.212,0.044
Gender,49.0%,48.0%,0.575,0.021
Careunit,78.9%,78.4%,0.732,0.014
Weight,79.50 (23.11),81.35 (24.69),0.035,0.077
Elix Score,8.82 (7.52),9.54 (7.67),0.006,0.096
Saps,19.80 (5.17),20.29 (5.42),0.008,0.092
Sofa,5.55 (3.50),5.99 (3.74),<0.001,0.123
Vent,49.6%,52.3%,0.123,0.055
Vaso,31.2%,32.7%,0.387,0.032


In [20]:
tab_wtd_fmt <- tab_wtd_df %>%
setNames(c("trtd", "ctrl", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_ = strsplit(cov, " ") %>% map_chr(first)) %>%
mutate(cov_fmt = str_replace_all(cov_, "_", " ")) %>%
mutate(cov_fmt = str_replace_all(cov_fmt, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_fmt = case_when(
    grepl("icd|vs|lab|day", cov_) ~ toupper(cov_fmt),
    TRUE ~ tools::toTitleCase(cov_fmt)
)) %>%
mutate(ctrl = ifelse(grepl("%", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse(grepl("%", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov_fmt, ctrl, trtd, pval) %>%
setNames(c("Covariate", "Echo (weighted cohort)", "Non-Echo (weighted cohort)",
           "p value (weighted cohort)"))

tab_wtd_fmt

Covariate,Echo (weighted cohort),Non-Echo (weighted cohort),p value (weighted cohort)
n,5842.36,5739.48,
Age,65.02 (16.83),65.92 (16.61),0.076
Gender,48.3%,49.2%,0.523
Careunit,79.4%,77.9%,0.178
Weight,82.70 (26.86),82.12 (25.47),0.163
Elix Score,9.90 (7.68),9.53 (7.60),0.114
Saps,22.18 (5.10),22.08 (5.49),0.134
Sofa,7.28 (3.81),6.99 (4.05),0.038
Vent,53.7%,50.7%,0.025
Vaso,33.6%,30.7%,0.020


In [21]:
tab_full_fmt <-
tab_fmt %>%
full_join(tab_wtd_fmt, by = "Covariate") %>%
full_join(tab_ps_fmt, by = "Covariate")

tab_full_fmt

Covariate,Non-Echo,Echo,p value,SMD,Echo (weighted cohort),Non-Echo (weighted cohort),p value (weighted cohort),Non-Echo (PS matching),Echo (PS matching),p value (PS matching),SMD (PS matching)
n,3262,3099,,,5842.36,5739.48,,1626,1626,,
Age,65.74 (16.55),66.69 (17.21),0.026,0.056,65.02 (16.83),65.92 (16.61),0.076,65.84 (17.07),66.58 (16.54),0.212,0.044
Gender,47.8%,50.8%,0.016,0.061,48.3%,49.2%,0.523,49.0%,48.0%,0.575,0.021
Careunit,79.5%,77.6%,0.067,0.047,79.4%,77.9%,0.178,78.9%,78.4%,0.732,0.014
Weight,83.17 (26.87),78.56 (23.58),<0.001,0.182,82.70 (26.86),82.12 (25.47),0.163,79.50 (23.11),81.35 (24.69),0.035,0.077
Elix Score,10.07 (7.67),8.51 (7.45),<0.001,0.207,9.90 (7.68),9.53 (7.60),0.114,8.82 (7.52),9.54 (7.67),0.006,0.096
Saps,20.76 (5.44),19.63 (5.79),<0.001,0.202,22.18 (5.10),22.08 (5.49),0.134,19.80 (5.17),20.29 (5.42),0.008,0.092
Sofa,6.33 (3.79),5.31 (3.62),<0.001,0.277,7.28 (3.81),6.99 (4.05),0.038,5.55 (3.50),5.99 (3.74),<0.001,0.123
Vent,58.3%,47.1%,<0.001,0.226,53.7%,50.7%,0.025,49.6%,52.3%,0.123,0.055
Vaso,37.6%,27.1%,<0.001,0.227,33.6%,30.7%,0.020,31.2%,32.7%,0.387,0.032


In [22]:
data.table::fwrite(tab_full_fmt, file.path(data_dir, "tableone_full.csv"))
data.table::fwrite(wtd_pval, file.path(data_dir, "wtd_pval.csv"))